In [2]:
from datetime import datetime, timedelta
import json
import uuid
import random 
from sqlalchemy import create_engine

from utils import reset_db, get_session, model_to_dict
from data.models import cultpass

# Udahub Accounts

## Cultpass Database

**Init DB**

In [3]:
cultpass_db = "data/external/cultpass.db"

In [3]:
reset_db(cultpass_db)

2025-12-14 17:34:26,762 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2025-12-14 17:34:26,766 INFO sqlalchemy.engine.Engine COMMIT
✅ Recreated data/external/cultpass.db with fresh schema


In [4]:
engine = create_engine(f"sqlite:///{cultpass_db}", echo=False)
cultpass.Base.metadata.create_all(engine)

**Experiences**

In [5]:
experience_data = []

with open('data/external/cultpass_experiences.jsonl', 'r', encoding='utf-8') as f:
    for line in f:
        experience_data.append(json.loads(line))

In [6]:
experience_data

[{'title': 'Carnival History Tour in Olinda',
  'description': "Discover the origins and vibrant traditions of Pernambuco's Carnival.",
  'location': 'Pernambuco, Brazil'},
 {'title': 'Sunset Paddleboarding',
  'description': 'Glide across calm waters at golden hour with all gear included.',
  'location': 'Santa Catarina, Brazil'},
 {'title': 'Pelourinho Colonial Walk',
  'description': 'Wander through colorful streets and learn about Afro-Brazilian history.',
  'location': 'Bahia, Brazil'},
 {'title': 'Samba Night at Lapa',
  'description': 'Dance the night away at a traditional samba club in the Lapa arches.',
  'location': 'Rio de Janeiro, Brazil'},
 {'title': 'Christ the Redeemer Experience',
  'description': 'Take a guided trip to one of the New Seven Wonders of the World with historical context.',
  'location': 'Rio de Janeiro, Brazil'},
 {'title': 'Modern Art at MASP',
  'description': 'Enjoy a guided visit to the São Paulo Museum of Art with insights into its top collections.',

In [7]:
with get_session(engine) as session:
    experiences = []

    for idx, experience in enumerate(experience_data):
        exp = cultpass.Experience(
            experience_id=str(uuid.uuid4())[:6],
            title=experience["title"],
            description=experience["description"],
            location=experience["location"],
            when=datetime.now() + timedelta(days=idx+1),
            slots_available=random.randint(1,30),
            is_premium=(idx % 2 == 0)
        )
        experiences.append(exp)

    session.add_all(experiences)

**User**

In [8]:
cultpass_users = []

with open('data/external/cultpass_users.jsonl', 'r', encoding='utf-8') as f:
    for line in f:
        cultpass_users.append(json.loads(line))

In [9]:
cultpass_users

[{'id': 'a4ab87',
  'name': 'Alice Kingsley',
  'email': 'alice.kingsley@wonderland.com',
  'is_blocked': True},
 {'id': 'f556c0',
  'name': 'Bob Stone',
  'email': 'bob.stone@granite.com',
  'is_blocked': False},
 {'id': '88382b',
  'name': 'Cathy Bloom',
  'email': 'cathy.bloom@florals.org',
  'is_blocked': False},
 {'id': '888fb2',
  'name': 'David Noir',
  'email': 'david.noir@shadowmail.com',
  'is_blocked': True},
 {'id': 'f1f10d',
  'name': 'Eva Green',
  'email': 'eva.green@ecosoul.net',
  'is_blocked': False},
 {'id': 'e6376d',
  'name': 'Frank Ocean',
  'email': 'frank.ocean@seawaves.io',
  'is_blocked': False}]

In [10]:
with get_session(engine) as session:
    db_users = []
    for user_info in cultpass_users:
        user = cultpass.User(
            user_id=user_info["id"],
            full_name=user_info["name"],
            email=user_info["email"],
            is_blocked=user_info["is_blocked"],
            created_at=datetime.now()
        )
        db_users.append(user)
    session.add_all(db_users) 

IntegrityError: (sqlite3.IntegrityError) UNIQUE constraint failed: users.email
[SQL: INSERT INTO users (user_id, full_name, email, is_blocked, created_at, updated_at) VALUES (?, ?, ?, ?, ?, CURRENT_TIMESTAMP), (?, ?, ?, ?, ?, CURRENT_TIMESTAMP), (?, ?, ?, ?, ?, CURRENT_TIMESTAMP), (?, ?, ?, ?, ?, CURRENT_TIMESTAMP), (?, ?, ?, ?, ?, CURRENT_TIMESTAMP), (?, ?, ?, ?, ?, CURRENT_TIMESTAMP) RETURNING updated_at, user_id]
[parameters: ('a4ab87', 'Alice Kingsley', 'alice.kingsley@wonderland.com', 1, '2025-12-29 17:01:57.415623', 'f556c0', 'Bob Stone', 'bob.stone@granite.com', 0, '2025-12-29 17:01:57.415789', '88382b', 'Cathy Bloom', 'cathy.bloom@florals.org', 0, '2025-12-29 17:01:57.415830', '888fb2', 'David Noir', 'david.noir@shadowmail.com', 1, '2025-12-29 17:01:57.415861', 'f1f10d', 'Eva Green', 'eva.green@ecosoul.net', 0, '2025-12-29 17:01:57.415888', 'e6376d', 'Frank Ocean', 'frank.ocean@seawaves.io', 0, '2025-12-29 17:01:57.415919')]
(Background on this error at: https://sqlalche.me/e/2

**Subscription**

In [11]:
with get_session(engine) as session:
    subscriptions = []
    for user_info in cultpass_users:
        subscription = cultpass.Subscription(
            subscription_id=str(uuid.uuid4())[:6],
            user_id=user_info["id"],
            status=random.choice(["active", "cancelled"]),
            tier=random.choice(["basic", "premium"]),
            monthly_quota=random.randint(2,10),
            started_at=datetime.now()
        )
        subscriptions.append(subscription)

    session.add_all(subscriptions)

IntegrityError: (sqlite3.IntegrityError) UNIQUE constraint failed: subscriptions.user_id
[SQL: INSERT INTO subscriptions (subscription_id, user_id, status, tier, monthly_quota, started_at, ended_at, created_at, updated_at) VALUES (?, ?, ?, ?, ?, ?, ?, CURRENT_TIMESTAMP, CURRENT_TIMESTAMP), (?, ?, ?, ?, ?, ?, ?, CURRENT_TIMESTAMP, CURRENT_TIMES ... 199 characters truncated ...  ?, ?, ?, ?, CURRENT_TIMESTAMP, CURRENT_TIMESTAMP) RETURNING created_at, updated_at, subscription_id]
[parameters: ('b31a64', 'a4ab87', 'cancelled', 'basic', 4, '2025-12-29 17:02:43.620196', None, '5cae27', 'f556c0', 'cancelled', 'premium', 6, '2025-12-29 17:02:43.620458', None, '0c7cfe', '88382b', 'active', 'basic', 7, '2025-12-29 17:02:43.620535', None, '8c91ce', '888fb2', 'active', 'basic', 3, '2025-12-29 17:02:43.620588', None, 'b794dd', 'f1f10d', 'cancelled', 'basic', 8, '2025-12-29 17:02:43.620638', None, '9db2cb', 'e6376d', 'cancelled', 'basic', 5, '2025-12-29 17:02:43.620684', None)]
(Background on this er

**Reservation**

In [12]:
# Applicable to `cultpass_users[0]` at the moment

with get_session(engine) as session:
    experience_ids = [
        exp.experience_id 
        for exp 
        in session.query(cultpass.Experience).all()
    ]

    reservation1 = cultpass.Reservation(
        reservation_id=str(uuid.uuid4())[:6],
        user_id=cultpass_users[0]["id"],
        experience_id=random.choice(experience_ids),
        status="reserved",
    )

    reservation2 = cultpass.Reservation(
        reservation_id=str(uuid.uuid4())[:6],
        user_id=cultpass_users[0]["id"],
        experience_id=random.choice(experience_ids),
        status="reserved",
    )

    session.add_all([reservation1, reservation2])

In [13]:
# TODO: Add more data
# Please notice that the reservations were set to first user only 
# If you want to simulate more users later, please create more reservations per user

with get_session(engine) as session:
    pass

In [13]:
experience_templates = []
with open('data/external/cultpass_experiences.jsonl', 'r', encoding='utf-8') as f:
    for line in f:
        experience_templates.append(json.loads(line))

with get_session(engine) as session:
    experiences = []

    for i in range(15):
        template = random.choice(experience_templates)
        exp_date = datetime.now() + timedelta(days=i+1, hours=random.randint(9, 20))

        exp = cultpass.Experience(
            experience_id=str(uuid.uuid4())[:8],
            title=template["title"],
            description=template["description"],
            location=template["location"],
            when=exp_date,
            slots_available=random.randint(0, 20),
            is_premium=random.choice([True, False])
        )
        experiences.append(exp)

    session.add_all(experiences)
    session.commit()

    all_experiences = session.query(cultpass.Experience).all()
    print(f"✅ Created {len(all_experiences)} Experiences")

✅ Created 29 Experiences


In [ ]:
# --- 4. Create Reservations ---
    # Create random reservations for ALL users, not just the first one
    reservations = []
    for user in cultpass_users_data:
        # Each user gets 0 to 3 reservations
        num_res = random.randint(0, 3)

        # Pick random experiences
        user_experiences = random.sample(all_experiences, min(num_res, len(all_experiences)))

        for exp in user_experiences:
            reservations.append(cultpass.Reservation(
                reservation_id=str(uuid.uuid4())[:8],
                user_id=user["id"],
                experience_id=exp.experience_id,
                status=random.choice(["reserved", "confirmed", "cancelled"]),
                created_at=datetime.now() - timedelta(days=random.randint(1, 5))
            ))

    session.add_all(reservations)
    print(f"✅ Created {len(reservations)} Reservations")

# --- Verification ---
with get_session(engine) as session:
    user_count = session.query(cultpass.User).count()
    sub_count = session.query(cultpass.Subscription).count()
    exp_count = session.query(cultpass.Experience).count()
    res_count = session.query(cultpass.Reservation).count()

    print("\n--- Final DB Stats ---")
    print(f"Users: {user_count}")
    print(f"Subscriptions: {sub_count}")
    print(f"Experiences: {exp_count}")
    print(f"Reservations: {res_count}")

In [14]:
# --- Verification ---
with get_session(engine) as session:
    user_count = session.query(cultpass.User).count()
    sub_count = session.query(cultpass.Subscription).count()
    exp_count = session.query(cultpass.Experience).count()
    res_count = session.query(cultpass.Reservation).count()

    print("\n--- Final DB Stats ---")
    print(f"Users: {user_count}")
    print(f"Subscriptions: {sub_count}")
    print(f"Experiences: {exp_count}")
    print(f"Reservations: {res_count}")


--- Final DB Stats ---
Users: 6
Subscriptions: 6
Experiences: 29
Reservations: 4


# Tests

In [15]:
with get_session(engine) as session:
    users = session.query(cultpass.User).all()
    for user in users:
        print(user)

<User(user_id='a4ab87', email='alice.kingsley@wonderland.com', is_blocked=True)>
<User(user_id='f556c0', email='bob.stone@granite.com', is_blocked=False)>
<User(user_id='88382b', email='cathy.bloom@florals.org', is_blocked=False)>
<User(user_id='888fb2', email='david.noir@shadowmail.com', is_blocked=True)>
<User(user_id='f1f10d', email='eva.green@ecosoul.net', is_blocked=False)>
<User(user_id='e6376d', email='frank.ocean@seawaves.io', is_blocked=False)>


In [16]:
with get_session(engine) as session:
    users = session.query(cultpass.User).all()
    for user in users:
        print(user.subscription)

<Subscription(subscription_id='7410df', user_id='a4ab87', status='cancelled', tier='basic')>
<Subscription(subscription_id='220896', user_id='f556c0', status='active', tier='premium')>
<Subscription(subscription_id='413ca9', user_id='88382b', status='active', tier='basic')>
<Subscription(subscription_id='c99595', user_id='888fb2', status='cancelled', tier='premium')>
<Subscription(subscription_id='2f0173', user_id='f1f10d', status='cancelled', tier='premium')>
<Subscription(subscription_id='9e4400', user_id='e6376d', status='cancelled', tier='basic')>


In [17]:
with get_session(engine) as session:
    experiences = session.query(cultpass.Experience).all()
    for experience in experiences:
        print(experience)

<Experience(experience_id='cd2251', title='Carnival History Tour in Olinda', when='2025-12-15 17:34:26.913576')>
<Experience(experience_id='0071be', title='Sunset Paddleboarding', when='2025-12-16 17:34:26.920269')>
<Experience(experience_id='816916', title='Pelourinho Colonial Walk', when='2025-12-17 17:34:26.920317')>
<Experience(experience_id='8e1f7f', title='Samba Night at Lapa', when='2025-12-18 17:34:26.920341')>
<Experience(experience_id='5568b0', title='Christ the Redeemer Experience', when='2025-12-19 17:34:26.920360')>
<Experience(experience_id='8d6a9f', title='Modern Art at MASP', when='2025-12-20 17:34:26.920380')>
<Experience(experience_id='c61eb8', title='Ibirapuera Park Bike Ride', when='2025-12-21 17:34:26.920396')>
<Experience(experience_id='037326', title='Carnival History Tour in Olinda', when='2025-12-30 17:01:41.820544')>
<Experience(experience_id='165486', title='Sunset Paddleboarding', when='2025-12-31 17:01:41.835519')>
<Experience(experience_id='434e7d', title=